In [6]:
!pip install tensorflow numpy matplotlib scikit-learn pillow pandas scikeras

INFO: pip is looking at multiple versions of scikeras to determine which version is compatible with other requirements. This could take a while.


In [7]:
!conda create -n tf-gpu --yes

!conda activate tf-gpu --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.5.0



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/tf-gpu



Preparing transaction: done
Verifying transaction: done
Executing transaction: done
#
# To activate this environment, use
#
#     $ conda activate tf-gpu
#
# To deactivate an active environment, use
#
#     $ conda deactivate


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after runn

In [3]:
!conda update -n base -c conda-forge conda

^C


In [4]:
!conda create -n tf-gpu --yes

!conda activate tf-gpu --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.5.0



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/tf-gpu



Preparing transaction: done
Verifying transaction: done
Executing transaction: done
#
# To activate this environment, use
#
#     $ conda activate tf-gpu
#
# To deactivate an active environment, use
#
#     $ conda deactivate


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after runn

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

from tensorflow.keras.applications import EfficientNetB0

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [11]:
# Define the data augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Validation data should not be augmented, only rescaled
test_datagen = ImageDataGenerator(rescale=1./255)

# Create the data generators
train_generator = train_datagen.flow_from_directory(
    'images/training',  # Path to your training data
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    'images/testing',  # Path to your validation data
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)



Found 2393 images belonging to 3 classes.
Found 597 images belonging to 3 classes.


In [12]:
def create_baseline_model(input_shape=(224, 224, 3), num_classes=3, learning_rate=0.001, l2_reg=0.001, activation='relu', optimizer='adam'):
    model = Sequential([
        Conv2D(32, (3, 3), activation=activation, input_shape=input_shape, kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation=activation, kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation=activation, kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(256, (3, 3), activation=activation, kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(256, activation=activation, kernel_regularizer=l2(l2_reg)),
        Dropout(0.5),
        Dense(128, activation=activation, kernel_regularizer=l2(l2_reg)),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    if optimizer == 'adam':
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    elif optimizer == 'adagrad':
        opt = tf.keras.optimizers.Adagrad(learning_rate=learning_rate)
    
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [13]:
model = KerasClassifier(model=create_baseline_model, input_shape=(224, 224, 3), num_classes=3)


In [14]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, min_delta=0.05, mode='max', verbose=1)


In [15]:
param_grid = {
    'model__learning_rate': [0.0001, 0.001, 0.01],
    'model__l2_reg': [0.0001, 0.001, 0.01],
    'model__activation': ['relu', 'tanh', 'sigmoid'],
    'model__optimizer': ['adam', 'sgd', 'rmsprop', 'adagrad'],
    'fit__epochs': [2],
    'fit__batch_size': [32, 64]
}


In [16]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, verbose=2, n_jobs=1)
grid_search_result = grid_search.fit(train_generator, validation_data=test_generator)


Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END fit__batch_size=32, fit__epochs=2, model__activation=relu, model__l2_reg=0.0001, model__learning_rate=0.0001, model__optimizer=adam; total time=  51.2s
[CV] END fit__batch_size=32, fit__epochs=2, model__activation=relu, model__l2_reg=0.0001, model__learning_rate=0.0001, model__optimizer=adam; total time=  48.0s


KeyboardInterrupt: 